In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Embedding
!pip install -q profanity-check
from profanity_check import predict

     |████████████████████████████████| 2.4MB 10.0MB/s 


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.20.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.svm.classes module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.svm. Anything that cannot be imp

In [4]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')

In [5]:
count = data['Genre'].value_counts()
print(count.head(10))

Rock          121404
Pop           108714
Metal          20291
Jazz           13545
Folk            8644
Indie           8449
R&B             2793
Hip-Hop         2240
Electronic      2213
Country         1890
Name: Genre, dtype: int64


In [6]:
df = data[data['Language']=='en']
df

,Artist,Song,Genre,Language,Lyrics
0,12 stones,world so cold,Rock,en,"It starts with pain, followed by hate\nFueled ..."
1,12 stones,broken,Rock,en,Freedom!\nAlone again again alone\nPatiently w...
2,12 stones,3 leaf loser,Rock,en,"Biting the hand that feeds you, lying to the v..."
3,12 stones,anthem for the underdog,Rock,en,You say you know just who I am\nBut you can't ...
4,12 stones,adrenaline,Rock,en,My heart is beating faster can't control these...
...,...,...,...,...,...
290178,bobby womack,i wish he didn t trust me so much,R&B,en,I'm the best friend he's got I'd give him the ...
290179,bad boys blue,i totally miss you,Pop,en,"Bad Boys Blue ""I Totally Miss You"" I did you w..."
290180,celine dion,sorry for love,Pop,en,Forgive me for the things That I never said to...
290181,dan bern,cure for aids,Indie,en,The day they found a cure for AIDS The day the...


In [7]:
options=['Pop']
df1 = data[data['Genre'].isin(options)]
df1 = pd.DataFrame(df, columns = ['Lyrics'])

In [ ]:
df1

,Lyrics
0,"It starts with pain, followed by hate\nFueled ..."
1,Freedom!\nAlone again again alone\nPatiently w...
2,"Biting the hand that feeds you, lying to the v..."
3,You say you know just who I am\nBut you can't ...
4,My heart is beating faster can't control these...
...,...
290178,I'm the best friend he's got I'd give him the ...
290179,"Bad Boys Blue ""I Totally Miss You"" I did you w..."
290180,Forgive me for the things That I never said to...
290181,The day they found a cure for AIDS The day the...


In [8]:
count=0
actual_count=0
for i in range(1000):
  #print(type(df1.iloc[i]['Lyrics']))
  if predict([df1.iloc[i]['Lyrics']])==1:
    #print("Offensive word found")
    count=count+1
    try:
      df1=df1.drop([i])
      actual_count=actual_count+1
    except:
      pass
  else:
    pass
print(len(df1))

249822


In [9]:
df1=df1.head(500)

In [10]:
pop_sub = df1

pop_sub.shape

(500, 1)

In [ ]:
pop_sub

,Lyrics
54770,lives in a little lonely town\nno one's around...
90090,"Break, break down, steady breakin' me on down\..."
253489,"All throughout the centuries, existing year to..."
224029,This tattered document\nA mystery you can solv...
37167,My love\nwherever you are\nWhatever you are\nD...
...,...
216222,We describe the sensation\nAs a tearing in our...
262616,hey\nhey\nhey\nhey\nhey\nhey\nhey\nhey\nhey\nh...
167508,Oh to sing this song makes me angry\nI'm not a...
141551,Watching the tidals that wash to the shore - I...


In [11]:
pop_lyrics = pop_sub['Lyrics'].astype('str')


In [12]:
poplist = list(pop_lyrics)

poplist[0]

"Freedom!\nAlone again again alone\nPatiently waiting by the phone\nHoping that you will call me home\nThe pain inside my love denied\nHopes and dreams swallowed by pride\nEverything I need it lies in you\n'Cause I'm broken\nI know I need you now\n'Cause deep inside I'm broken\nYou see the way I live\nI know I know your heart is broken\nWhen I turn away\nI need to be broken\nTake the pain away\nI question why you chose to die\nWhen you knew your truth I would deny\nYou look at me\nThe tears begin to fall\nAnd all in all faith is blind\nBut I fail time after time\nDaily in my sin I take your life\n'Cause I'm broken\nI know I need you now\n'Cause deep inside I'm broken\nYou see the way I live\nI know I know your heart is broken\nWhen I turn away\nI need to be broken\nTake the pain away\nAll the hate deep inside\nSlowly covering my eyes\nAll these things I hide\nAway from you again\nAll this fear holding me\nMy heart is cold and I believe\nNothing's gonna change\nUntil I'm broken\nThis ti

In [13]:
import string

def clean_text(txt):
    txt = "".join(i for i in txt if i not in string.punctuation).lower()
    txt = txt.replace("\n"," ")
    return txt 


In [14]:
corpus = [clean_text(x) for x in poplist]
corpus[0]

'freedom alone again again alone patiently waiting by the phone hoping that you will call me home the pain inside my love denied hopes and dreams swallowed by pride everything i need it lies in you cause im broken i know i need you now cause deep inside im broken you see the way i live i know i know your heart is broken when i turn away i need to be broken take the pain away i question why you chose to die when you knew your truth i would deny you look at me the tears begin to fall and all in all faith is blind but i fail time after time daily in my sin i take your life cause im broken i know i need you now cause deep inside im broken you see the way i live i know i know your heart is broken when i turn away i need to be broken take the pain away all the hate deep inside slowly covering my eyes all these things i hide away from you again all this fear holding me my heart is cold and i believe nothings gonna change until im broken this time let go cause im broken i know i need you now c

In [ ]:
len(corpus)

500

In [15]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

In [16]:
inp_sequences, total_words = get_sequence_of_tokens(corpus)

In [ ]:
total_words

6416

In [17]:
from keras.preprocessing.sequence import pad_sequences

max_sequence_len = 500
print(max_sequence_len)
inp_sequences = np.array(pad_sequences(inp_sequences,
                       maxlen = max_sequence_len, padding='pre'))

500


In [ ]:
print(inp_sequences.shape)
inp_sequences

(123751, 500)


array([[   0,    0,    0, ...,    0,  844,  147],
       [   0,    0,    0, ...,  844,  147,   73],
       [   0,    0,    0, ...,  147,   73,   73],
       ...,
       [   0,    0,    0, ..., 1339,    5,   42],
       [   0,    0,    0, ...,    5,   42,  207],
       [   0,    0,    0, ...,   42,  207,  623]], dtype=int32)

In [18]:
predictors, label = inp_sequences[:,:-1],inp_sequences[:,-1]

In [19]:
import keras.utils as ku

label = ku.to_categorical(label, num_classes=total_words)

In [ ]:
label.shape

(123751, 6416)

In [20]:
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Embedding, LSTM
from keras.callbacks import EarlyStopping
model = Sequential()
model.add(Embedding(total_words, 50, input_length=400))
# Add a LSTM Layer
model.add(LSTM(100)) 
model.add(Dense(total_words/2, activation='relu'))  
# In the last layer, the shape should be equal to the total number of words present in our corpus
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')  
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 400, 50)           320800    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               60400     
_________________________________________________________________
dense (Dense)                (None, 3208)              324008    
_________________________________________________________________
dense_1 (Dense)              (None, 6416)              20588944  
Total params: 21,294,152
Trainable params: 21,294,152
Non-trainable params: 0
_________________________________________________________________


In [ ]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1)
mod_test = model.fit(predictors, label, epochs= 10, callbacks=[es])

Epoch 1/10
3868/3868 [==============================] - 377s 90ms/step - loss: 6.1976 - accuracy: 0.0548
Epoch 2/10
3868/3868 [==============================] - 347s 90ms/step - loss: 4.9880 - accuracy: 0.1423
Epoch 3/10
3868/3868 [==============================] - 347s 90ms/step - loss: 4.1441 - accuracy: 0.2323
Epoch 4/10
 178/3868 [>.............................] - ETA: 5:30 - loss: 3.4866 - accuracy: 0.3007

In [ ]:
!pip install -q pyyaml h5py

model.save('pop01.h5')

In [ ]:
def make_lyrics(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],
                     maxlen=max_sequence_len-1,padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    print(seed_text)

In [ ]:
print (make_lyrics("They look for", 50))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


They look for me im a tangled puppet and i might be a mess but i sure can survive find myself awake counting sad days 1 2 3 thats too many for too cool for dawn jump a wonderful country but the man hes burning it down yea yea yea yea yea yea
None
